In [1]:
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
# import sys
# import os
# sys.path.append(os.path.realpath('.'))
import pandas as pd
from scipy.spatial import distance
import scipy as stats
from matplotlib import pyplot
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics
from sklearn.metrics import mean_squared_error
from datetime import datetime as dt
from math import sqrt
from peaks import *
from data import *
import psycopg2
import matplotlib.pyplot as plt

In [2]:
path = '/Users/angeloenrique/Dev/puentes/plataformashm/ModeloAR/Datos/'
df = pd.read_parquet(path + 'AC05-30_05_2021_dataset.parquet')

In [33]:
fmt = '%Y-%m-%d %H:%M:%S'
tstamp1 = dt.strptime('2016-04-06 21:26:27', fmt)
tstamp2 = dt.strptime('2016-04-09 09:06:02', fmt)

In [38]:
tstamp1

datetime.datetime(2016, 4, 6, 21, 26, 27)

In [43]:
(tstamp2 - tstamp1).days

2

In [42]:
td.days

2

In [3]:
df2 = df.reset_index()

In [4]:
df2

,timestamp,x,y,z
0,2021-05-30 00:00:00-04:00,-0.097533,-0.157881,1.203469
1,2021-05-30 00:00:00.009000-04:00,0.153806,0.184945,0.976033
2,2021-05-30 00:00:00.017000-04:00,0.629866,-0.504777,1.153120
3,2021-05-30 00:00:00.025000-04:00,-0.112412,0.234619,1.771191
4,2021-05-30 00:00:00.034000-04:00,-0.262747,-0.578065,0.922212
...,...,...,...,...
10367892,2021-05-30 23:59:59.959000-04:00,0.093516,0.136901,1.121871
10367893,2021-05-30 23:59:59.967000-04:00,-0.172702,-0.082963,1.456946
10367894,2021-05-30 23:59:59.975000-04:00,0.126402,0.124686,1.091488
10367895,2021-05-30 23:59:59.984000-04:00,-0.132769,-0.094364,1.331945


In [5]:
df2[df2['y']==0.0]

,timestamp,x,y,z


In [43]:
try:
    t = np.array(df2['t'])
except:
    t = np.array(0)

In [44]:
if t.any(): print('si')

In [38]:
df3 = df2[df2['timestamp'].dt.hour == 4]

In [41]:
df3.reset_index(drop=True)

,timestamp,x,y,z
0,2021-05-30 04:00:00-04:00,0.218012,0.232991,1.042875
1,2021-05-30 04:00:00.009000-04:00,-0.044290,-0.064235,1.299827
2,2021-05-30 04:00:00.017000-04:00,0.034010,-0.071564,1.666154
3,2021-05-30 04:00:00.025000-04:00,-0.357489,-0.215695,0.998602
4,2021-05-30 04:00:00.034000-04:00,-0.207935,-0.119608,1.411807
...,...,...,...,...
431995,2021-05-30 04:59:59.959000-04:00,0.133449,0.133644,1.088014
431996,2021-05-30 04:59:59.967000-04:00,-0.074045,-0.248268,1.518580
431997,2021-05-30 04:59:59.975000-04:00,0.218012,-0.072377,1.057631
431998,2021-05-30 04:59:59.984000-04:00,-0.144514,-0.138335,1.359722


In [2]:
data = getParquetDay(0)

In [3]:
data

,timestamp,x,y,z
0,2021-05-30 00:00:00-04:00,-0.097533,-0.157881,1.203469
1,2021-05-30 00:00:00.009000-04:00,0.153806,0.184945,0.976033
2,2021-05-30 00:00:00.017000-04:00,0.629866,-0.504777,1.153120
3,2021-05-30 00:00:00.025000-04:00,-0.112412,0.234619,1.771191
4,2021-05-30 00:00:00.034000-04:00,-0.262747,-0.578065,0.922212
...,...,...,...,...
431995,2021-05-30 00:59:59.959000-04:00,0.034010,-0.007231,1.159197
431996,2021-05-30 00:59:59.967000-04:00,-0.614310,-0.084591,1.244270
431997,2021-05-30 00:59:59.975000-04:00,0.247767,0.343737,0.994261
431998,2021-05-30 00:59:59.984000-04:00,-0.149211,-0.142408,1.422226


In [10]:
def applyCoef(serie):
    model = []
    for i in range(3):
        model.append(AutoReg(serie[i], lags = [1,2,3,4,5,6], old_names=True))
    model_fit = []
    for i in range(3):
        model_fit.append(model[i].fit())
    coef = [x.params for x in model_fit]
    return coef

In [11]:
day = dt(2021, 5, 30, 00, 00)

In [12]:
date = str(day.day).zfill(2) + '_' + str(day.month).zfill(2) + '_' + str(day.year)

In [13]:
date

'30_05_2021'

In [23]:
sensor = 'AC05'

In [24]:
for hora in range(24):
    try:
        data = getParquetDay(sensor, date, hora)
    except OSError as e:
        print('Error not found', e)
        continue
    time = data['timestamp']


    x = np.array(data['x'])
    y = np.array(data['y'])
    z = np.array(data['z'])
    series_x = scpa(time, x, 3, 60)
    series_y = scpa(time, y, 3, 60)
    series_z = scpa(time, z, 3, 60)
    sens_x = []
    sens_y = []
    sens_z = []
    for i in range(0,3):
        sens_x.append(np.array(x[series_x[i][0]:series_x[i][-1]]))
        sens_y.append(np.array(y[series_y[i][0]:series_y[i][-1]]))
        sens_z.append(np.array(z[series_z[i][0]:series_z[i][-1]]))
    x_coef = applyCoef(sens_x)
    y_coef = applyCoef(sens_y)
    z_coef = applyCoef(sens_z)
    print("x coef = ", x_coef)
    print("y coef = ", y_coef)
    print("z coef = ", z_coef)


x coef =  [array([-0.01225898, -0.19339159,  0.08930809,  0.00122294, -0.08644962,
        0.18795107, -0.27259551]), array([-0.01337005, -0.20518396,  0.08498907,  0.01269706, -0.08699671,
        0.2080904 , -0.24405852]), array([-0.0123554 , -0.21434548,  0.09035754,  0.00143816, -0.09223825,
        0.16902995, -0.28013276])]
y coef =  [array([ 0.03168287, -0.19956583,  0.08875203,  0.02805696, -0.07860169,
        0.15742606, -0.24182313]), array([ 0.02355774, -0.20958449,  0.07288822,  0.01628241, -0.08933961,
        0.16542397, -0.23323053]), array([ 0.02719268, -0.18884826,  0.07732468,  0.01738849, -0.07951694,
        0.15412163, -0.22603121])]
z coef =  [array([ 1.61414992, -0.21181463,  0.0792946 , -0.0029458 , -0.10103934,
        0.18474455, -0.25768512]), array([ 1.51813534, -0.19680485,  0.1039232 , -0.00383743, -0.10515192,
        0.20453691, -0.23317692]), array([ 1.52101206, -0.18423334,  0.10088233, -0.00607181, -0.09490389,
        0.20385561, -0.25439583])]
x co